In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Wed Nov 30 12:44:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |      

In [ ]:
%%writefile vectorAdd.cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int *a, int *b, int *c) {
  *c = *a + *b;
}

int main() {
  int a, b, c;

  // host copies of variables a, b & c
  int *d_a, *d_b, *d_c;

  // device copies of variables a, b & c
  int size = sizeof(int);

  // Allocate space for device copies of a, b, c
  cudaMalloc((void **)&d_a, size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);

  // Setup input values  
  c = 0;
  a = 3;
  b = 5;

  // Copy inputs to device
  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

  // Launch add() kernel on GPU
  add<<<1,1>>>(d_a, d_b, d_c);

  // Copy result back to host
  cudaError err = cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);
  if(err!=cudaSuccess) {
      printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
  }

  printf("result is %d\n",c);

  // Cleanup
  cudaFree(d_a);
  cudaFree(d_b);
  cudaFree(d_c);
  return 0;
}

Writing vectorAdd.cu


In [ ]:
!nvcc vectorAdd.cu
!ls
!./a.out

a.out  sample_data  vectorAdd.cu
result is 8


In [ ]:
!ls /usr/local/cuda*

/usr/local/cuda:
bin		   EULA.txt  libnvvp	       nvvm-prev  src
compat		   extras    nsightee_plugins  README	  targets
compute-sanitizer  include   nvml	       samples	  tools
DOCS		   lib64     nvvm	       share	  version.json

/usr/local/cuda-11:
bin		   EULA.txt  libnvvp	       nvvm-prev  src
compat		   extras    nsightee_plugins  README	  targets
compute-sanitizer  include   nvml	       samples	  tools
DOCS		   lib64     nvvm	       share	  version.json

/usr/local/cuda-11.2:
bin		   EULA.txt  libnvvp	       nvvm-prev  src
compat		   extras    nsightee_plugins  README	  targets
compute-sanitizer  include   nvml	       samples	  tools
DOCS		   lib64     nvvm	       share	  version.json


In [ ]:
!cp -rf /usr/local/cuda-11/samples/1_Utilities/deviceQuery ./deviceQuery

In [ ]:
!ls ./deviceQuery

deviceQuery.cpp  Makefile  NsightEclipse.xml  readme.txt


In [ ]:
!nvcc -I/usr/local/cuda-11/samples/common/inc deviceQuery/deviceQuery.cpp -o a.out

In [ ]:
!./a.out

./a.out Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          11.2 / 11.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15110 MBytes (15843721216 bytes)
  (40) Multiprocessors, ( 64) CUDA Cores/MP:     2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shared mem

In [ ]:
!wget http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2

--2022-11-30 12:45:31--  http://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Resolving www.cs.virginia.edu (www.cs.virginia.edu)... 128.143.67.11
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2 [following]
--2022-11-30 12:45:32--  https://www.cs.virginia.edu/~skadron/lava/Rodinia/Packages/rodinia_3.1.tar.bz2
Connecting to www.cs.virginia.edu (www.cs.virginia.edu)|128.143.67.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360636246 (344M) [application/x-tar]
Saving to: ‘rodinia_3.1.tar.bz2’

rodinia_3.1.tar.bz2 100%[===================>] 343.93M  12.1MB/s    in 30s     

2022-11-30 12:46:03 (11.3 MB/s) - ‘rodinia_3.1.tar.bz2’ saved [360636246/360636246]



In [ ]:
!tar -xf rodinia_3.1.tar.bz2

In [ ]:
!gcc --version

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!env

NV_LIBCUBLAS_DEV_VERSION=11.4.1.1043-1
NV_CUDA_COMPAT_PACKAGE=cuda-compat-11-2
__EGL_VENDOR_LIBRARY_DIRS=/usr/lib64-nvidia:/usr/share/glvnd/egl_vendor.d/
NV_CUDNN_PACKAGE_DEV=libcudnn8-dev=8.1.1.33-1+cuda11.2
PYDEVD_USE_FRAME_EVAL=NO
LD_LIBRARY_PATH=/usr/lib64-nvidia
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.8.4-1+cuda11.2
TCLLIBPATH=/usr/share/tcltk/tcllib1.19
CLOUDSDK_PYTHON=python3
LANG=en_US.UTF-8
NV_LIBNPP_DEV_PACKAGE=libnpp-dev-11-2=11.3.2.152-1
ENABLE_DIRECTORYPREFETCHER=1
HOSTNAME=4bbb15ba839f
OLDPWD=/
CLOUDSDK_CONFIG=/content/.config
USE_AUTH_EPHEM=1
KMP_EXTRA_ARGS=--listen_host=172.28.0.2 --target_host=172.28.0.2 --tunnel_background_save_url=https://colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-s-1jfirqarstn8z --tunnel_background_save_delay=10s --tunnel_periodic_background_save_frequency=30m0s --enable_output_coalescing=true --output_coalescing_required=true
NV_LIBNPP_VERSION=11.3.2.152-1
NV_NVPROF_DEV_PACKAGE=cuda-nvprof-11-2=11.2.152-1
NVIDIA_VI

#### Particle Filter

##### OpenMP

In [ ]:
!cd rodinia_3.1/openmp/particlefilter && make openmp
!ls rodinia_3.1/openmp/particlefilter 

In [ ]:
!./rodinia_3.1/openmp/particlefilter/particle_filter -x 1024 -y 1024 -z 10 -np 100000 | tail --lines=2

PARTICLE FILTER TOOK 23.232613
ENTIRE PROGRAM TOOK 24.234400


##### CUDA

In [ ]:
!cd rodinia_3.1/cuda/particlefilter && make all
!ls rodinia_3.1/cuda/particlefilter 

In [ ]:
!./rodinia_3.1/cuda/particlefilter/particlefilter_naive -x 1024 -y 1024 -z 10 -np 100000 | tail --lines=2

PARTICLE FILTER TOOK 1.325444
ENTIRE PROGRAM TOOK 2.426719


In [ ]:
!./rodinia_3.1/cuda/particlefilter/particlefilter_float -x 1024 -y 1024 -z 10 -np 100000 | tail --lines=2

PARTICLE FILTER TOOK 1.145506
ENTIRE PROGRAM TOOK 2.279990


#### LU-Decomposition

OpenMP

In [ ]:
!cd rodinia_3.1/openmp/lud && make lud_omp
!ls rodinia_3.1/openmp/lud 

cd omp; make
make[1]: Entering directory '/content/rodinia_3.1/openmp/lud/omp'
make[1]: 'lud_omp' is up to date.
make[1]: Leaving directory '/content/rodinia_3.1/openmp/lud/omp'
ls: cannot access '2': No such file or directory
4071235 -rwxr-xr-x 1 599459 21 44506945 Dec 11  2015 ./rodinia_3.1/data/lud/2048.dat

rodinia_3.1/openmp/lud:
total 32
4070316 drwxr-xr-x 2 599459 21 4096 Dec 11  2015 base
4070313 drwxr-xr-x 2 599459 21 4096 Nov 30 13:40 common
4070320 -rwxr-xr-x 1 599459 21  166 Dec 11  2015 Makefile
4070308 drwxr-xr-x 2 599459 21 4096 Nov 30 13:40 omp
4070321 -rwxr-xr-x 1 599459 21  416 Dec 11  2015 README
4070322 -rwxr-xr-x 1 599459 21   71 Nov 30 13:40 run
4070307 -rw-r--r-- 1 599459 21   31 Dec 11  2015 run_offload
4070303 drwxr-xr-x 2 599459 21 4096 Dec 11  2015 tools


In [ ]:
!./rodinia_3.1/openmp/lud/omp/lud_omp -n 2 -s 4096

Generate input matrix internally, size =4096
Creating matrix internally size=4096
running OMP on host
Time consumed(ms): 5103.015000


##### CUDA

In [ ]:
!cd rodinia_3.1/cuda/lud && make lud_cuda
!ls rodinia_3.1/cuda/lud 

cd cuda; make
make[1]: Entering directory '/content/rodinia_3.1/cuda/lud/cuda'
make[1]: Nothing to be done for 'all'.
make[1]: Leaving directory '/content/rodinia_3.1/cuda/lud/cuda'
base  common  cuda  Makefile  README  run  tools


In [ ]:
!./rodinia_3.1/cuda/lud/cuda/lud_cuda -s 4096

WG size of kernel = 32 X 32
Generate input matrix internally, size =4096
Creating matrix internally size=4096
Time consumed(ms): 186.069000


#### Needleman-Wunch

##### OpenMP

In [ ]:
!cd rodinia_3.1/openmp/nw && make needle
!ls rodinia_3.1/openmp/nw 

g++ -g -O3 -fopenmp needle.cpp -o needle 
Makefile  needle  needle.cpp  run  run_offload


In [ ]:
!./rodinia_3.1/openmp/nw/needle 30000 10 2

tcmalloc: large alloc 3600244736 bytes == 0x55f9c193e000 @  0x7fea8e3c51e7 0x55f9bf60181e 0x55f9bf600b09 0x7fea8dd93c87 0x55f9bf600b3a
tcmalloc: large alloc 3600244736 bytes == 0x55fa982b4000 @  0x7fea8e3c51e7 0x55f9bf60182b 0x55f9bf600b09 0x7fea8dd93c87 0x55f9bf600b3a
Start Needleman-Wunsch
Num of threads: 2
Processing top-left matrix
Processing bottom-right matrix
Total time: 4169.683 ms


##### CUDA

In [ ]:
!cd rodinia_3.1/cuda/nw && make debug
!ls rodinia_3.1/cuda/nw 

/usr/local/cuda/bin/nvcc  -g needle.cu -o needle -I/usr/local/cuda/include -L/usr/local/cuda/lib64 
Makefile	 needle     needle.h	      README
Makefile_nvidia  needle.cu  needle_kernel.cu  run


In [ ]:
!nvprof ./rodinia_3.1/cuda/nw/needle 30000 10 

WG size of kernel = 16 
tcmalloc: large alloc 3600244736 bytes == 0x55e2956c6000 @  0x7f6dea0e31e7 0x55e293007930 0x55e2930077a1 0x7f6de9114c87 0x55e29300757a
tcmalloc: large alloc 3600244736 bytes == 0x55e36c03c000 @  0x7f6dea0e31e7 0x55e29300794f 0x55e2930077a1 0x7f6de9114c87 0x55e29300757a
tcmalloc: large alloc 3600244736 bytes == 0x55e4429b2000 @  0x7f6dea0e31e7 0x55e29300796e 0x55e2930077a1 0x7f6de9114c87 0x55e29300757a
Start Needleman-Wunsch
==1608== NVPROF is profiling process 1608, command: ./rodinia_3.1/cuda/nw/needle 30000 10
Processing top-left matrix
Processing bottom-right matrix
Total time: 224.265 ms
==1608== Profiling application: ./rodinia_3.1/cuda/nw/needle 30000 10
==1608== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   56.03%  2.34024s         1  2.34024s  2.34024s  2.34024s  [CUDA memcpy DtoH]
                   37.88%  1.58227s         2  791.14ms  776.22ms  806.05ms  [CUDA memcpy HtoD]
      